In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import seisbench.data as sbd
from torch.utils.data import Dataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score
import time

In [25]:
# Device Checking
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [26]:
# Loading the dataset
dataset = sbd.STEAD()

# Label Encoding simple
label_map = {"noise": 0, "earthquake_local": 1}
dataset.metadata["label"] = dataset.metadata["trace_category"].map(label_map)

# Spliting the dataset
train_indices = dataset.metadata[dataset.metadata["split"] == "train"].index
dev_indices = dataset.metadata[dataset.metadata["split"] == "dev"].index

# 20% sanity mode (Using 20% of the dataset only)
train_indices = train_indices[:int(0.2 * len(train_indices))]

print("Training samples:", len(train_indices))
print("Validation samples:", len(dev_indices))

2026-03-01 11:58:20,884 | seisbench | WARNING | Output component order not specified, defaulting to 'ZNE'.


Training samples: 215161
Validation samples: 63283


In [27]:
class SteadTorchDataset(Dataset):
    def __init__(self, dataset, indices, crop_len=3000):
        self.dataset = dataset
        self.indices = indices
        self.crop_len = crop_len

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        wf = self.dataset.get_waveforms([real_idx])[0]

        # Crop for speed
        wf = wf[:, :self.crop_len]

        # Normalize
        wf = wf - wf.mean(axis=1, keepdims=True)
        wf = wf / (wf.std(axis=1, keepdims=True) + 1e-6)

        label = self.dataset.metadata.iloc[real_idx]["label"]

        return torch.tensor(wf, dtype=torch.float32), \
               torch.tensor(label, dtype=torch.float32)


train_dataset = SteadTorchDataset(dataset, train_indices)
dev_dataset = SteadTorchDataset(dataset, dev_indices)

train_loader = DataLoader(train_dataset,
                          batch_size=16,
                          shuffle=True,
                          num_workers=2,
                          pin_memory=True)

dev_loader = DataLoader(dev_dataset,
                        batch_size=16,
                        shuffle=False,
                        num_workers=2,
                        pin_memory=True)

In [28]:
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size,
                              padding=0, dilation=dilation)

    def forward(self, x):
        x = F.pad(x, (self.padding, 0))
        return self.conv(x)

In [29]:
class ResidualBlock(nn.Module):
    def __init__(self, channels, kernel_size, dilation):
        super().__init__()
        self.conv1 = CausalConv1d(channels, channels, kernel_size, dilation)
        self.norm1 = nn.GroupNorm(1, channels)
        self.conv2 = CausalConv1d(channels, channels, kernel_size, dilation)
        self.norm2 = nn.GroupNorm(1, channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        residual = x
        out = self.relu(self.norm1(self.conv1(x)))
        out = self.dropout(self.norm2(self.conv2(out)))
        out += residual
        return self.relu(out)

In [30]:
class ResidualCausalTCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_conv = nn.Conv1d(3, 32, kernel_size=1)

        dilations = [1,2,4,8,16,32,64,128]  # 8 blocks

        self.blocks = nn.Sequential(
            *[ResidualBlock(32, 5, d) for d in dilations]
        )

        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        x = self.input_conv(x)
        x = self.blocks(x)
        x = self.pool(x).squeeze(-1)
        return self.fc(x)


model = ResidualCausalTCN().to(device)

In [31]:
labels = dataset.metadata.loc[train_indices]["label"].values
weights = compute_class_weight("balanced",
                               classes=np.unique(labels),
                               y=labels)

pos_weight = torch.tensor([weights[0] / weights[1]]).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


In [32]:
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=1e-4,
                              weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
# scaler = torch.amp.GradScaler(device_type='cuda')

In [33]:
scaler = torch.cuda.amp.GradScaler()

def train_one_epoch():
    model.train()
    total_loss = 0

    for x, y in train_loader:
        x = x.to(device, non_blocking=True)
        y = y.unsqueeze(1).to(device, non_blocking=True)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(x)
            loss = criterion(outputs, y)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    return total_loss / len(train_loader)


def evaluate():
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x, y in dev_loader:
            x = x.to(device)
            y = y.unsqueeze(1).to(device)

            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item()

            preds = (torch.sigmoid(outputs) > 0.5).cpu().numpy()
            all_preds.extend(preds.flatten())
            all_labels.extend(y.cpu().numpy().flatten())

    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)

    return total_loss / len(dev_loader), precision, recall

/tmp/ipykernel_8038/1242697417.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    start = time.time()

    train_loss = train_one_epoch()
    val_loss, precision, recall = evaluate()
    scheduler.step()

    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Epoch Time: {time.time() - start:.2f} sec")
    print("-"*40)


/tmp/ipykernel_8038/1242697417.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


In [ ]:
model.eval()
dummy = torch.randn(1,3,3000).to(device)

with torch.no_grad():
    start = time.time()
    for _ in range(100):
        _ = model(dummy)
    end = time.time()

print("Avg inference time per window:",
      (end-start)/100, "seconds")